# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing Turi Create functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up Turi Create

Make sure you have the latest version of Turi Create.

In [ ]:
pip install turicreate

     |████████████████████████████████| 92.0 MB 4.6 kB/s 
     |████████████████████████████████| 322 kB 64.4 MB/s 
     |████████████████████████████████| 3.5 MB 53.6 MB/s 
     |████████████████████████████████| 86.4 MB 56 kB/s 
     |████████████████████████████████| 3.6 MB 47.2 MB/s 
     |████████████████████████████████| 18.3 MB 68 kB/s 
     |████████████████████████████████| 2.9 MB 16.6 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 449 kB 49.1 MB/s 
     |████████████████████████████████| 3.8 MB 57.5 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=4b5349340328b493d44204f8cefa8f03e09af4c473b488a1cb5fae2236c2388f
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320860 sha256=6473fed00e1

In [ ]:
from __future__ import division
import turicreate
import math
import string

# Data preparation

**1.** We will use a dataset consisting of baby product reviews on Amazon.com.

In [ ]:
from google.colab import drive
drive.mount ('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# products = turicreate.SFrame('amazon_baby.sframe/')

products = turicreate.SFrame('/content/gdrive/My Drive/Turicreate/Week 13/amazon_baby.sframe/')

Now, let us see a preview of what the dataset looks like.

In [ ]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [ ]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

**2.** Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [ ]:
import string 
def remove_punctuation(text):
    try: # python 2.x
        text = text.translate(None, string.punctuation) 
    except: # python 3.x
        translator = text.maketrans('', '', string.punctuation)
        text = text.translate(translator)
    return text

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = turicreate.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [ ]:
products[269]['word_count']

{'a': 1.0, 'favorite': 1.0, 'house': 1.0, 'in': 1.0, 'our': 1.0}

## Extract sentiments

**3.** We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [ ]:
products = products[products['rating'] != 3]
len(products)

166752

**4.** Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [ ]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'highly': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'this':1.0, 'for': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0,'headachesthanks': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'2995': 1.0, 'for': 1.0,'barnes': 1.0, 'at': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'postits': 1.0, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

**5.** Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [ ]:
train_data, test_data = products.random_split(.8, seed=1)
print(len(train_data))
print(len(test_data))

133416
33336


# 7. Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [ ]:
sentiment_model = turicreate.logistic_classifier.create(train_data,
                                                        target = 'sentiment',
                                                        features=['word_count'],
                                                        validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.778177  | 5.227135     | 0.942638          |

| 3         | 12       | 2.778177  | 5.793856     | 0.967822          |

| 4         | 13       | 2.778177  | 6.362386     | 0.976495          |

| 5         | 14       | 2.778177  | 6.922968     | 0.976495          |

+-----------+----------+-----------+--------------+-------------------+

In [ ]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 5
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 4.723

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[offsi]              : 21.7657
word_count[conclusions]        : 21.6818
word_count[torsional]          : 21.6818
word_count[geeeezzz]           : 21.6818
word_count[inversely]          : 21.6818

Lowest Negative Coefficients
----------------------------
word_count[wahwah] 

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [ ]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

In [ ]:
weights

name,index,class,value,stderr
(intercept),None,1,0.7797362814630173,None
word_count,recommend,1,0.3052457737558048,None
word_count,highly,1,0.9060636221511342,None
word_count,disappointed,1,-2.426792494486643,None
word_count,love,1,0.8405057320631645,None
word_count,it,1,0.010479446044391189,None
word_count,planet,1,-0.3846729911256599,None
word_count,and,1,0.03631437278677283,None
word_count,bags,1,0.13057598098929268,None
word_count,wipes,1,-0.002075180317769144,None


**8.**There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [ ]:
num_positive_weights = (weights['value'] >= 0).sum()
num_negative_weights = (weights['value'] < 0).sum()

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 91073 
Number of negative weights: 30640 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

**9.** Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [ ]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'again': 1.0, 'book':1.0, 'same': 1.0, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'peeling': 1.0, '5':1.0, 'about': 1.0, 'f ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'on': 1.0, 'waste': 1.0,'wouldnt': 1.0, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [ ]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [ ]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

**10.** We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using Turi Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [ ]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print(scores)

[4.788907309204979, -3.000782222421493, -8.188501360764482]


### Predicting sentiment

**11.** These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [ ]:
def class_predictions(scores):
    """ make class predictions
    """
    preds = []
    for score in scores:
        if score > 0:
            pred = 1
        else:
            pred = -1
        preds.append(pred)
    return preds

In [ ]:
class_predictions(scores)

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from Turi Create.

In [ ]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data))

Class predictions according to Turi Create:
[1, -1, -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from Turi Create.

### Probability predictions

**12.** Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [ ]:
def calculate_proba(scores):
    """ Calculate the probability predictions from the scores.
    """
    proba_preds = []
    for score in scores:
        proba_pred =  1 / (1 + math.exp(-score))
        proba_preds.append(proba_pred)
    return proba_preds

calculate_proba(scores)

[0.9917471313286145, 0.047390547488973575, 0.0002777527712167853]

**Checkpoint**: Make sure your probability predictions match the ones obtained from Turi Create.

In [ ]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data, output_type='probability'))

Class predictions according to Turi Create:
[0.9917471313286145, 0.047390547488973575, 0.0002777527712167852]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

**13.** We now turn to examining the full test dataset, **test_data**, and use Turi Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [ ]:
# probability predictions on test_data using the sentiment_model
test_data['proba_pred'] = sentiment_model.predict(test_data, output_type='probability')
test_data

name,review,rating,word_count,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'postits': 1.0, ...",1
Nature's Lullabies FirstYear Sticker Calendar ...,"I love this littlecalender, you can keep ...",5.0,"{'too': 1.0, 'stickers':1.0, 'illustrations': ...",1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5.0,"{'reference': 1.0,'have': 1.0, 'out': 1.0, ...",1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,"{'typical': 1.0, 'your':1.0, 'the': 1.0, ...",1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5.0,"{'farm': 1.0, 'out': 1.0,'say': 1.0, 'again': ...",1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5.0,"{'own': 1.0, 'his': 1.0,'on': 1.0, 'a': 1.0, ...",1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2.0,"{'kenzie': 1.0, 'my':1.0, 'down': 1.0, 'gi ...",-1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5.0,"{'book': 1.0, 'elmo':1.0, 'love': 1.0, 'for': ...",1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5.0,"{'pictures': 1.0,'personalization': 1.0, ...",1


In [ ]:
test_data['name','proba_pred'].topk('proba_pred', k=20).print_rows(20)

+-------------------------------+------------+
|              name             | proba_pred |
+-------------------------------+------------+
| Maxi-Cosi Pria 70 with Tin... |    1.0     |
| Mamas &amp; Papas 2014 Urb... |    1.0     |
| Barin Toys Teething Pendan... |    1.0     |
| CTA Digital 2-in-1 iPotty ... |    1.0     |
| Alva Baby One Size Washabl... |    1.0     |
| Graco FastAction Fold Jogg... |    1.0     |
| Britax Boulevard 70-G3 Con... |    1.0     |
| The Original CJ's BuTTer (... |    1.0     |
| Diono RadianRXT Convertibl... |    1.0     |
| Diono RadianRXT Convertibl... |    1.0     |
| Britax Decathlon Convertib... |    1.0     |
| Quinny 2012 Buzz Stroller,... |    1.0     |
| Roan Rocco Classic Pram St... |    1.0     |
| Baby Einstein Around The W... |    1.0     |
| Britax Frontier Booster Ca... |    1.0     |
| Evenflo X Sport Plus Conve... |    1.0     |
| P'Kolino Silly Soft Seatin... |    1.0     |
| Peg Perego Aria Light Weig... |    1.0     |
| Fisher-Pric

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


**14.** Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [ ]:
test_data['name','proba_pred'].topk('proba_pred', k=20, reverse=True).print_rows(20)

+-------------------------------+------------------------+
|              name             |       proba_pred       |
+-------------------------------+------------------------+
| Luna Lullaby Bosom Baby Nu... | 3.229790837176558e-63  |
| The First Years True Choic... | 1.632282318641265e-24  |
| Jolly Jumper Arctic Sneak ... |  8.11031137680362e-20  |
| Motorola MBP36 Remote Wire... | 7.797281600633468e-16  |
| VTech Communications Safe ... | 1.8411614785278732e-14 |
| Fisher-Price Ocean Wonders... | 6.345094942114933e-14  |
| Levana Safe N'See Digital ... | 6.578528511903017e-14  |
| Safety 1st High-Def Digita... | 1.5408410008559129e-13 |
| Snuza Portable Baby Moveme... | 6.301835284300002e-13  |
| Adiri BPA Free Natural Nur... | 9.314560911105397e-13  |
| Samsung SEW-3037W Wireless... | 5.9206029667152455e-12 |
| Motorola Digital Video Bab... |  5.98620084618247e-12  |
| Cloth Diaper Sprayer--styl... | 1.0091402074340469e-11 |
| Munchkin Nursery Projector... | 3.4551832742296466e-11

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

**15.** We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [ ]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    num_correct = sum(predictions == true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = num_correct/len(data)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [ ]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9221862251019919

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

**16.** There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [ ]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [ ]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [ ]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [ ]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [ ]:
print(train_data[0]['word_count'])

{'recommend': 1.0, 'highly': 1.0, 'disappointed': 1.0, 'love': 1.0, 'it': 3.0, 'planet': 1.0, 'and': 3.0, 'bags': 1.0, 'wipes': 1.0, 'not': 2.0, 'early': 1.0, 'came': 1.0, 'i': 1.0, 'does': 1.0, 'wise': 1.0, 'my': 2.0, 'was': 1.0, 'now': 1.0, 'wipe': 1.0, 'holder': 1.0, 'leak': 1.0, 'keps': 1.0, 'osocozy': 1.0, 'moist': 1.0}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [ ]:
print(train_data[0]['word_count_subset'])

{'disappointed': 1.0, 'love': 1.0}


## Train a logistic regression model on a subset of data

**17.** We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [ ]:
simple_model = turicreate.logistic_classifier.create(train_data,
                                                     target = 'sentiment',
                                                     features=['word_count_subset'],
                                                     validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 1.248081     | 0.866815          |

| 6         | 7        | 1.463653     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.4976

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [ ]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

**18.** Now, we will inspect the weights (coefficients) of the **simple_model**:

In [ ]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.299544955202694,0.01208885413305366
word_count_subset,disappointed,1,-2.3550925006107213,0.050414988855697916
word_count_subset,love,1,1.3654354936790394,0.030354629510904802
word_count_subset,well,1,0.5042567463979325,0.021381300630990332
word_count_subset,product,1,-0.32055549299557207,0.015431132136201484
word_count_subset,loves,1,1.677271455559297,0.04823282753835112
word_count_subset,little,1,0.5206286360250221,0.02146914756649044
word_count_subset,work,1,-0.6217000124253117,0.023033059794584844
word_count_subset,easy,1,1.1936618983284684,0.029288869202029347
word_count_subset,great,1,0.9446912694798406,0.02095099265905039


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [ ]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+----------------------+
|        name       |    index     | class |        value         |
+-------------------+--------------+-------+----------------------+
| word_count_subset |    loves     |   1   |  1.677271455559297   |
| word_count_subset |   perfect    |   1   |  1.514486267027139   |
| word_count_subset |     love     |   1   |  1.3654354936790394  |
|    (intercept)    |     None     |   1   |  1.299544955202694   |
| word_count_subset |     easy     |   1   |  1.1936618983284684  |
| word_count_subset |    great     |   1   |  0.9446912694798406  |
| word_count_subset |    little    |   1   |  0.5206286360250221  |
| word_count_subset |     well     |   1   |  0.5042567463979325  |
| word_count_subset |     able     |   1   | 0.19143830229475226  |
| word_count_subset |     old      |   1   | 0.08539618866781937  |
| word_count_subset |     car      |   1   | 0.058834990068021176 |
| word_count_subset |     less     |   1   | -0.

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [ ]:
simple_weights = simple_model.coefficients
positive_significant_words = simple_weights[(simple_weights['value'] > 0) & (simple_weights['name'] == "word_count_subset")]['index']
#print len(positive_significant_words)
len(positive_significant_words)
positive_significant_words

dtype: str
Rows: 10
['love', 'well', 'loves', 'little', 'easy', 'great', 'able', 'perfect', 'old', 'car']

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [ ]:
weights.filter_by(positive_significant_words, 'index')

name,index,class,value,stderr
word_count,love,1,0.8405057320631645,None
word_count,well,1,0.4010755749238933,None
word_count,loves,1,0.9749823125149905,None
word_count,little,1,0.4099316272590633,None
word_count,easy,1,0.7349826255685794,None
word_count,great,1,0.7789532883814279,None
word_count,able,1,0.10752802191587835,None
word_count,perfect,1,1.0447994204060929,None
word_count,old,1,0.07967490900991046,None
word_count,car,1,0.11965787650846046,None


# Comparing models

**19.** We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [ ]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.976494573364514

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [ ]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

**20.** Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [ ]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9221862251019919

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [ ]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

**21.** It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [ ]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [ ]:
print ((test_data['sentiment'] == +1).sum())

print ((test_data['sentiment'] == -1).sum())

28095
5241


In [ ]:
print ((test_data['sentiment'] == +1).sum()/len(test_data['sentiment']))

0.8427825773938085


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?